Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

# Read the main CSV file and save to a main dataframe

In [14]:
import pandas as pd
import numpy as np

#read the main dataframe from the csv
mdf = pd.read_csv("bank_marketing.csv")

# Clients CSV


## Fix the boolean columns and clean data

In [15]:
# pull the nessary columns to a seprate dataframe
client = mdf[["client_id",
             "age","job",
             "marital",
             "education",
             "credit_default",
             "mortgage"]].copy()
# Check for nulls and unique values
# client.info()
# client["credit_default"].unique()

# Define a mapping dictionary (ps. decided to convert unkown to false cause it's more safe than making it true)
mapping_dict = {'unknown': False, 'no': False, 'yes': True}

# Map to True and False to convert to boolean
client["credit_default"] = client["credit_default"].map(mapping_dict).astype(bool)

# Check for unique values
# client["mortgage"].unique()

# Map to True and False to convert to boolean
client["mortgage"] = client["mortgage"].map(mapping_dict).astype(bool)

# Change "." to "_" in jobs
client["job"] = client["job"].str.replace('.', '_')

# Change "." to "_" in education and "unknown" to np.NaN
client["education"] = client["education"].str.replace('.', '_')
client["education"] = client["education"].replace('unknown', np.NaN)

## Save to CSV

In [16]:
# Save dataframe without index
client.to_csv("client.csv", index=False)

# Campaign CSV

## Make the last_contact_date

In [17]:
# pull the nessary columns to a seprate dataframe
campaign = mdf[["client_id",
               "number_contacts",
               "contact_duration",
               "previous_campaign_contacts",
               "previous_outcome",
               "campaign_outcome"]].copy()

# Fix the month coulmn in the main dataframe

# Map month names to numeric values
month_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

mdf['month'] = mdf['month'].map(month_mapping)

# Add a new column named 'year' and fill it with 2022
mdf['year'] = 2022

# Concatenate year, month, and day columns into a single string column
mdf['date_str'] = mdf['year'].astype(str) + '-' + mdf['month'].astype(str) + '-' + mdf['day'].astype(str)

# Convert the new string column to datetime
campaign['last_contact_date'] = pd.to_datetime(mdf['date_str'], format='%Y-%m-%d')

# Drop the intermediate string column if needed
mdf = mdf.drop(columns=['date_str'])

## Fix the boolean Columns

In [18]:
# Check for nulls and unique values
# campaign.info()
# campaign["previous_outcome"].unique()

prev_map = {'nonexistent':False, 'failure':False, 'success':True}
campaign["previous_outcome"] = campaign["previous_outcome"].map(prev_map).astype(bool)

# Check for unique values
# campaign["campaign_outcome"].unique()

campaign["campaign_outcome"] = campaign["campaign_outcome"].map({'no': False, 'yes': True}).astype(bool)

## Save to CSV

In [19]:
# Save dataframe without index
campaign.to_csv("campaign.csv", index=False)

# Economics CSV

## Make the economics DataFrame

In [20]:
# pull the nessary columns to a seprate dataframe
economics = mdf[["client_id",
                "cons_price_idx",
                "euribor_three_months"]].copy()

# Check for nulls and data types
# economics.info()

## Save to CSV

In [21]:
# Save dataframe without index
economics.to_csv("economics.csv", index=False)